In [58]:
import Random

##KNOWN ISSUES
##PERIODIC BOUNDARY - CRASH WHILE CHANGING LANE TO THE RIGHT WHILE PERSON FROM BEGINNING OF AN ARRAY CHANGES LANE ALSO
##TEMPORARY FIX - NO CHANGING LANES FROM INDEX 1-5

In [64]:
function mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
    """
    Performs 1 MCS. Velocity updates happen in parallel for every car.
    """
    
    update_velocities_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
    
    print_state(vehicles, velocities, lane_changes)
    
    update_positions_closed(vehicles, velocities, lane_changes)
    
end

mcs_closed (generic function with 1 method)

In [59]:
function update_velocities_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
    """
    Agent behaviour checklist 
    ## SAFELY CHANGE LANE TO THE RIGHT
    ## ACCELERATE
    ## SAFELY TRY TO PASS
    ## TRY A-HOLE PASS LEFT (NOT DONE YET)
    ## TRY A-HOLE PASS RIGHT (NOT DONE YET)
    If no move was chosen, agent has to slow down to nearest person in front.
  
    """
    
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    next_vehicle=fill(0, (lines, 1))
    
    
    for i = 1:length
        for j = 1:lines
            if next_vehicle[j]==0 && vehicles[j,i]!=0
                next_vehicle[j] = i + length
            end
        end
    end
    for j = 1:lines
        if next_vehicle[j]==0
            next_vehicle[j]=length+10 ##NIE MA NIKOGO W STANIE POCZATKOWYM
        end
    end
        
    for i = length:-1:1
        for j = lines:-1:1
            if vehicles[j, i]!=0
                move="none"
                ### SAFE LANE CHANGE - RIGHT
                if j>1 && velocities[j,i]>1 && i>4
                    situation=check_right_mirror(vehicles, velocities, next_vehicle, j, i)
                    if situation=="safe"
                        move="change_lane_right"
                    end
                end
                
                ### ACCELERATION
                if move=="none"
                    if next_vehicle[j]>i+velocities[j,i]
                        move="accelerate"
                    end
                end
                
                ### SAFE LANE CHANGE - LEFT
                if move=="none" && j<lines && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ## WYPRZEDZANIE
                        if next_vehicle[j+1]>i+velocities[j,i]
                            situation=check_left_mirror(vehicles, velocities, next_vehicle, j, i)
                            if situation=="safe"
                                move="change_lane_left"
                            end
                        end
                    end
                end
                
                ### SLOWING DOWN
                if move=="none"
                    if next_vehicle[j]<=i+velocities[j,i]
                        move="slow_down"
                    end
                end
                        
                      
                if move == "accelerate"
                   accelerate(velocities, i, j, next_vehicle[j], max_velocity)
                   slow_down_random(velocities, i, j, p)
                end
                
                if move == "change_lane_right"
                    lane_changes[j,i]=-1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j-1]=i+floor(velocities[j,i]/2)
                end
                
                if move == "change_lane_left"
                    lane_changes[j,i]=+1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j+1]=i+floor(velocities[j,i]/2)
                end
                        
                if move == "slow_down"
                   slow_down_car(velocities, i, j, next_vehicle[j]) 
                   slow_down_random(velocities, i, j, p)
                end
                        
                next_vehicle[j] = i 
                
#                 if j>1 && next_vehicle[j-1]>i+velocities[j,i]+1 && vehicles[j-1, i]==0 && i>4 && velocities[j,i]>1
#                     lane_changes[j,i]=-1
#                     if next_vehicle[j-1] > i + velocities[j, i]
#                         accelerate(velocities, i, j, next_vehicle[j-1], max_velocity)
#                     else
#                         slow_down_car(velocities, i, j, next_vehicle[j-1])  
#                     end
#                     slow_down_random(velocities, i, j, p)
#                     next_vehicle[j-1]=i + ceil(velocities[j,i]/2)    
#                 else
#                     if next_vehicle[j] > i + velocities[j, i]
#                     ##HAS PLACE - ACCELERATING
#                         accelerate(velocities, i, j, next_vehicle[j], max_velocity)
#                         slow_down_random(velocities, i, j, p)  
#                     else
#                         #### WYPRZEDZANIE
#                         if j<lines && next_vehicle[j]<next_vehicle[j+1] && i>4
#                             ### ADDITIONAL CONDITION: Nastepny pojazd jedzie wolniej od naszego
#                             if next_vehicle[j]>length
#                                 pos_next_vehicle=next_vehicle[j]%length
#                             else
#                                 pos_next_vehicle=next_vehicle[j]
#                             end
#                             if velocities[j, pos_next_vehicle]<velocities[j,i]
#                                 lane_changes[j,i]=1
#                                 if next_vehicle[j+1] > i + velocities[j, i]
#                                     accelerate(velocities, i, j, next_vehicle[j+1], max_velocity)
#                                     slow_down_random(velocities, i, j, p)
#                                     next_vehicle[j+1] = i + ceil(velocities[j,i]/2)
#                                 else
#                                     slow_down_car(velocities, i, j, next_vehicle[j+1])
#                                     slow_down_random(velocities, i, j, p)
#                                     next_vehicle[j+1] = i + ceil(velocities[j,i]/2)    
#                                 end
#                             end
#                         else 
#                             slow_down_car(velocities, i, j, next_vehicle[j])
#                             slow_down_random(velocities, i, j, p)      
#                         end
#                     end
#                 end                
            end
        end
    end
end

function update_positions_closed(vehicles, velocities, lane_changes)
        """
        
        """
    length = size(velocities)[2]
    lines = size(velocities)[1]
    
    # If rightmost vehicle goes through periodic boundary, avoid updating it second time by marking new position as "already checked"
    # At the beginning avoid_pos holds zeroes.
    avoid_pos=fill(0, (lines, length))
    
    # To avoid crashes occuring from passing the periodic boundary, we add to our diagram vehicles from front at the last possible moment,
    temp_update_vel=fill(0, (lines, 5))
    temp_update_veh=fill(0, (lines, 5))
    old_state_veh=copy(vehicles)
    old_state_vel=copy(velocities)
    for i = length:-1:1
        for j = lines:-1:1
            if (vehicles[j, i] != 0) && (velocities[j, i] != 0) && (avoid_pos[j,i]==0)
                if (i+velocities[j, i]) > length
                    new_pos=(i+velocities[j, i])%length
                    lane_change=lane_changes[j, i]
                    #if vehicles[j+lane_change, new_pos] == 1
                        #println("Car from $j, $i crashed into $(j+lane_change), $(i+velocities[j, i]), $lane_change, $(velocities[j,i])")
                        #println("CRASH REPORT")
                        #print_state(vehicles, velocities, lane_changes)
                    #end
                    temp_update_vel[j+lane_change, new_pos] = velocities[j, i]
                    temp_update_veh[j+lane_change, new_pos] = 1
                    ##vehicles[j+lane_change, new_pos] = 1
                    ##velocities[j+lane_change, new_pos] = velocities[j, i]
                    ##avoid_pos[j+lane_change, new_pos]=1
                    vehicles[j, i] = 0
                    velocities[j, i] = 0
                    lane_changes[j, i] = 0
                else
                    lane_change=lane_changes[j, i]
                    if vehicles[j+lane_change, (i+velocities[j, i])] == 1
                        println("Car from $j, $i crashed into $(j+lane_change), $(i+velocities[j, i]), $lane_change, $(velocities[j,i])")
                        println("CRASH REPORT")
                        print_state(vehicles, velocities, lane_changes)
                    end
                    vehicles[j+lane_change, (i+velocities[j, i])] = 1
                    velocities[j+lane_change, (i+velocities[j, i])] = velocities[j, i]
                    vehicles[j, i]=0
                    velocities[j, i]=0
                    lane_changes[j, i] = 0
                end
            end
        end
    end
    for i = 5:-1:1
        for j = lines:-1:1
            if temp_update_veh[j,i]!=0
                if vehicles[j, i] == 1
                    print_state(old_state_veh, old_state_vel, lane_changes)
                    println("Cars crashed on $j, $i")
                    println("CRASH REPORT")
                    print_state(vehicles, velocities, lane_changes)
                    print_state(temp_update_veh, temp_update_vel, lane_changes)
                end
                vehicles[j,i]=temp_update_veh[j,i]
                velocities[j,i]=temp_update_vel[j,i]
                temp_update_vel[j,i]=0
                temp_update_veh[j,i]=0
            end
        end
    end
end

update_positions_closed (generic function with 1 method)

In [4]:
function check_right_mirror(vehicles, velocities, next_vehicle, row, pos)
    """
    Function checks if we are able to change lane.
    First - check if next vehicle is further than our "range"
    Then - check every space from our goal, up to (X - now 4) places back.
    
    """
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    if velocities[row, pos]>0
        flag=0
        if next_vehicle[row-1]<pos+velocities[row,pos]
            flag=1
        end
        for p = pos-4:pos+velocities[row,pos]
            if p<1
                tp=p+length
            elseif p>length
                tp=p%length
            else
                tp=p    
            end
            if vehicles[row-1, tp]!=0
                flag=1
            end
        end
        if flag==0
            return "safe"
        else
            return "unsafe"
        end
    end
end

function check_left_mirror(vehicles, velocities, next_vehicle, row, pos)
    """
    Function checks if we are able to change lane.
    First - check if next vehicle is further than our "range"
    Then - check every space from our goal, up to (X - now 4) places back.
    """
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    if velocities[row, pos]>0
        flag=0
        if next_vehicle[row+1]<pos+velocities[row,pos]
            flag=1
        end
        for p = pos-4:pos+velocities[row,pos]+1
            if p<1
                tp=p+length
            elseif p>length
                tp=p%length
            else
                tp=p    
            end
            if vehicles[row+1, tp]!=0
                flag=1
            end
        end
        if flag==0
            return "safe"
        else
            return "unsafe"
        end
    end
end

check_left_mirror (generic function with 1 method)

In [5]:
# try to accelerate
function accelerate(velocities, pos, row, next_vehicle, max_velocity)
    """
    If velocity is smaller than max_velocity chceck if collision occurs.
    If no collision => accelerate
    """
    if velocities[row,pos] < max_velocity
        if pos + velocities[row,pos] + 1 < next_vehicle
            velocities[row,pos] += 1
        end
    end
end


# slow down due to next car
function slow_down_car(velocities, pos, row, next_vehicle)
    """
    Slow down to land on position (next_vehicle - 1) after move.
    """
    velocities[row,pos] = next_vehicle - pos - 1
end


# slow down randomly
function slow_down_random(velocities, pos, row, p)
    """
    Random event of slowing down with probability p. 
    Velocity cannot drop below 0.
    """
    if Random.rand() < p
        if velocities[row,pos] > 0
            velocities[row,pos] -= 1
        end
    end
end

slow_down_random (generic function with 1 method)

In [74]:
##MAIN PROGRAM
length = 110
lines = 4
p = 0.2
max_velocity = 5 
min_velocity = 0

0

In [45]:
function print_state(vehicles, velocities, lane_changes)
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    for j in lines:-1:1
        str = ""
        for i = 1:length
            if vehicles[j,i] == 0
                str = str * "."
            else
                str = str * string(velocities[j,i])
            end
        end
        println(str)
    end
    str=""
    for i = 1:length
        str=str*"="
    end
    println(str)
end

function licz_auta(vehicles, aut)
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    s=0
    for j in lines:-1:1 
        for i = 1:length
            if vehicles[j,i]!=0
                s+=1
            end
        end
    end
    if s!=aut
        println("ZŻARŁO")
    end
    println(s)
    return s
end

licz_auta (generic function with 1 method)

In [75]:
# closed boundary -  multiline

velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

no_cars=length
no_steps=1000
aut=no_cars
vehicles[1, 1:no_cars] .=1
Random.shuffle!(vehicles)
#print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
end
aut=licz_auta(vehicles, aut)

..01..0......1.0..1.1...1.1....01..1.00.........1...1.......1.1..1.1...............001.........1.......01..1..
....................001..1...1.1.1..1..1......01.1.1..00..........1.01.1.......0....0.1.1.01.1..001..1..1.....
0....1.0.1..1.1...1...............1.......1..1...1.1.1.1.1.1.................00..01.01..1...1.1.1..01...01...0
.1..00......1.......0.........1.1....1..0.....1..1.0..1.....1.1...1.1...01.....1.............1................
..1.1.1.......01...1.2...1.2...1.2..001..........2...2.......1.2..1.2..............01.1.........2......1.2..2.
....................01.1..2...1.0.2..2..2.....1.1.1.0.01...........00.1.2......0....1..0.01.0.1.01.1..2..1....
1.....01..2..0.2...2...............2.......2..1...1.1.1.1.1.2................01..1.00.1..2...1.1.1.0.2..1.2..0
..1.01.......2......1..........1.2....1.0......2..01...2.....1.1...1.1..1.2.....2.............2...............
1..1.1.2......1.2...2..2..1..2..2..000.2...........3...3......1..0.1..3............0.0.2..........3.....2..2..
.

001...3...01.0.1.2.......4......001.2..3...4.....1.2..0.2..3.....4....01.2..3...4....4....5.....2...2...4....0
.01...2..3......5......1.01.00001...2.....3.....3...1.2..2..3....4....5.....2...4....5.....4......5.....3...2.
....5.....5...........2..000.1.2.......3....4........4....5........3...4....5........3...4.......5.........5..
.....4....4....5.........5.....1..0.0.2.....01.1...3...4....5.....1.2..3....3.....2...1.0001.2..2..3....3...3.
00.2.....00.01..2..3.........2..01.2..3...4....2..2..01...3...4......01.2..3...4....3....5.....1..3...2.....00
01.2....3...4........2..00.00001.2....3......4.....1.1..2..3...4.....4.....1..3.....5.....4....5.......3...2..
..5......5.....5........0001..2..3........4.....5........5.....4......4....5.....4......4....5........5.......
.4.......4....5.....5.........1.1.1.1...3...1.0.2.....3....5.....0.1..3...4....4....2..0001.2..2..2...4....3..
01...2...01.1.2...3...4........01.2..2...4....1..2..01.1.....4....1..1.2..3...4....3...4......1.2....2..3...00
1

..5......4....1..2...3...0.1.1..3...3.........5.........5.....4....000.2.....2..01.0.2..3...4......2...4......
.....3......3...2...01.0.2....3.....01.0.1.01......3..2...4.........5.....2..2....0.2....2..00001.0001.2.....4
4....2..1.0001.1.1.1.1....4....3....1.01.2...2.......5.......4....2...4.......5......2..1.0001...1.2..3...3...
....5..........3...01.0.1.1.0...3............................5........5......2...4....4............3...4....4.
.......4.....0.2...3....01..1.2....3...3...........5..4......4....0001...3.....01.01...3...4....4....2.....5..
...4....4......1..1.1.01...2.....2..1.01..01.2..........2.....5..........2..2..1..1...3....00001.0001.2..2....
....2..1.0001.1.1.1.1.2.......3...2..01.2..3...3..........5......2..3.....4........3...1.0000.1...2..2...3...4
..5......5........01.01..1.01......4....................5.........5........3...2.....4....5...........3....4..
..5........1.1...3....1.1.2..2..2.....3...4...............5......0001.2.....2..0.01.2.....4....4....2..2......
.

..01.2..3......2..3...001.01.1.2...2..3......4....5...........5........0001.2...1.1.01..2..2...4.....1..0001.2
........3...001.1.01.2..3...3........4....5.......5.....5.......5....................3....5.......3....5.....5
..4.....3...1.0...1.2....3...2....4....4.....5.....5.....5......4......4...5..........3...4........2..3...4...
..5.....2..1.2....3....01.001.001.2..2...3.....5.........5........5.......2..3...3.1.0001.0.2..2...2..3...4...
.01.2..3...4.....3...001.01.1.2..3...3...4.......5.....5...........3...001.1..1..1.01.2...1..3.....1..1.001.2.
....5......000.1.01.2..3...3...3.........5.....4.......5.....5.......5.....4............4......4.....4......5.
4.....4....0.01....2..3.....1..3......4....5......5.....5.....5.....5...........4...4....4....4......2...4....
5......2..1.2..3.....1.0.001.001.1..2..2....4.......5.........5........4....3...3...0001.01...2..3...2...4....
01.2..3...3....3....001.01.1.2..3...2...4....5........4.....5.........001.1.2..2..01.2..2..2....3...2..001.2..
.

.....0001.00001..2..3...4....2..2....3......5.....2..1.1.2..0.01.1..3...2..3.....2..01..1.01.2..3...2..3....4.
.01.2..2...3...2...1.2...2..3....4....5...........5........5.........5......4.......5..........5......4....1.1
01.2......2..3......5........5.....5.........5...............5.....4....01........5........5.....4.....2...0.0
..2...00.0001.01.1.2....3...4....5.......5.......3....4.........1.001....0.2..2..3...3....4....2....2..1.2..3.
..2..001.00000.2...3...3....2..2..3.....3........2..1.1.1..21.1.1.2....2..2...4....01.2..01.2..3...2..2...4...
01.2..1..3....2..1..1..3.......4.....5.....5....................4.........5.....4........4..........5.....1.1.
1.1..3......3...3........0.3......5.....5.........5....4..........4....00.2............4........4....3...1.1.0
.2..1.01.001.01.0.2..3.....4....5.....5.......5.....4.....1..2...001.2...1...2..3...3...4.....2..3....1.2..3..
3...001.000001...3....3...3...2..3...4.....4.......1.1.1.1...1.1.2..3....2..3.....00.2..00.2..3...2..2..3.....
1

...

.....5.....5......4.....5.....5......5......5........5......5.....5........5........4..................5.....5
.........4.......3....5.......4......4....5.......5.......5.......5......5.......4......4.......5.....5.....4.
..1.2.....4.....01.01.01.000001.....3......5..........1.001...1.1..01.1.001..1.1.1.2.........4....5.......5...
1......4......01.00.01...00000...3...4....3...2..1..001.001..2..1.2..00.1.01.1.2..1.1..2....4....2..1..01..000
....4.....5.....5.....4......5.....5......5......5........5......5.....5........5.......5...................4.
..5..........5......4......4......5......5.....5.......4.......5.......4......4......5...............5.....4..
.1.2..2.......1.1.01.00.000001.2.......3........5......000.1...0.1.1.1.001.2..1.1.2..3......4....4.....5......
.2.........1..1.000.1.2..00000......3....3...2..1.1.01.001.1...1.2..001..01.0.2..1.1.2...3......2..0.1.1.1.001
..5.....4......5.....4....4.......5.....5......5......5........5......5.....5........5.......5................
.

2..0.01.1.1.00..0.2..3....1.1....00.01.2...1..0001..1.01.1.01..0.1..3....2..2..2..1..01.2..2....2..1.2..3...0.
........5......4....4....5......5.....5............5.....4.......5.....5........5...........5.......5......5..
...5........4.....3....5.....5......3......5.....5.......5.......4......5......5.......5.....5........5.....4.
.00.1.2...2..3...2..2........4....5.....1.01.1..001....1.1....2..01.1.1.2....3.....3....2..2..2...4....2..01.0
..00.1.1.0.001..1...2...2..0.2...01.0.2..2..1.001.2..01.0.01.1.1..2....3...1..2..1.1.1.2..2..2....1.2..3...01.
..5..........4.....4....5.....5......5.....5............4....5........5.....5........4...........5.......4....
..5.....5.......3....3......5.....4....4........5.....5.......5......5.......5......4.......5.....5........3..
.00..2..3...3...2..2..3..........5.....1.01.1.1.01.1....1.2.....01.1.1.2..3.....4.....3...2..2..3.....1..01.00
..01..1.01.01.2..2....3...01...0.1.01...2..1.001.2..00.00.1.1.1.2...3.....0.2...0.1.1.2..2..2..3...2..3...01.2
.

Excessive output truncated after 524363 bytes.

............5......5.......4......4......5........4.....3...3...3...3.......4....5.......2...4.......5.......5
.3...3....4......5......2....1.01.1.01.2..1..3...000.2..1.2...2......5.....5........00.1.01.1.1.1.001.2.....2.
2...3...4.....3...2..0000..01.1.1.0.1.0.00001.01..2..3...2..0...2....2..2..3......1.1.2..0001.1.1..1.001.1.2..
..5......................5......4...............5.....4......5.......5......5.......4........5......5.........
....4....5.......5......5......5......5.......5.......4....3...2...3...4........5.....4....3.....5........5...
3...3...4.....5.......3...3...01.1.01.2..1.2....0001...0.2..3...3.........5.....3...01..01.1.1.0.001.2..3.....
..3....4....4....2..00001..1.1.0.00..01.0001.01.1...3...2..00.....3....2..3...3....1.2..0001.1.1.2..001.1.2..2
.......5......................4.....5................4....5.......5.......4......5......5.........4......5....
.4......4.....5.......5......4......5......5.......5......3...2..3....4....4.........4....3...3.......5.......
.

110

In [14]:
velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

no_steps=100
scenario_no=5
load_scenario(vehicles, velocities, lane_changes, scenario_no)
print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
end

..................................................4.............................
..................................................00............................
.................................................1..............................
..................................................5.............................
..................................................01............................
.................................................2..............................
.......................................................5........................
..................................................1.2...........................
...................................................3............................
............................................................5...................
...................................................1..3.........................
......................................................4.........................
............................

.....................5........5........5........................................
................................................................................
..................................5.............................................
..........................5........5........5...................................
................................................................................
.......................................5........................................
...............................5........5........5..............................
................................................................................
............................................5...................................
....................................5........5........5.........................
................................................................................
.................................................5..............................
............................

................................................................................
................................................5...............................
.......................................4......5.........5.......................
................................................................................
.....................................................5..........................
...........................................5.......5.........5..................
................................................................................
..........................................................5.....................
................................................5.......5.........5.............
................................................................................
...............................................................5................
.....................................................5.......5.........5........
............................

................................5.......5..........5............................
................................................................................
............................................5...................................
.....................................5.......5..........5.......................
................................................................................
.................................................5..............................
..........................................5.......5..........5..................


In [13]:
function load_scenario(vehicles, velocities, lane_changes, no)
    if no == 1
        vehicles[2, 1]=1
        velocities[2, 1]=5
        vehicles[1, 3]=1
        velocities[1,3]=0
        vehicles[2, 11]=1
        velocities[2, 11]=5
        vehicles[1, 13]=1
        velocities[1,13]=0
    end
    if no==2
        vehicles[1, 1]=1
        velocities[1, 1]=5
        vehicles[1, 3]=1
        velocities[1,3]=2
        vehicles[1, 9]=1
        velocities[1,9]=3
        vehicles[1, 14]=1
        velocities[1,14]=1
        vehicles[2,1]=1
        velocities[2,1]=5
        vehicles[2,9]=1
        velocities[2,9]=2
        vehicles[2,7]=1
        velocities[2,7]=2
        vehicles[2,5]=1
        velocities[2,5]=2
        
    end
    if no==3
        vehicles[2, 1]=1
        velocities[2, 1]=5
        vehicles[1, 4]=1
        velocities[1, 4]=2
        vehicles[1, 6]=1
        velocities[1, 6]=1
        vehicles[1, 76]=1
        velocities[1, 76]=5
        vehicles[2, 76]=1
        velocities[2, 76]=5
    end
    if no==4
        vehicles[2, 80]=1
        velocities[2, 80]=1
        vehicles[1, 80]=1
        velocities[1, 80]=1
        vehicles[1, 2]=1
        velocities[1, 2]=0
        vehicles[2, 2]=1
        velocities[2, 2]=0
        vehicles[1, 3]=1
        velocities[1, 3]=1
        vehicles[2, 3]=1
        velocities[2, 3]=1
    end
    if no==5
        vehicles[2, 52]=1
        velocities[2, 52]=0
        vehicles[2, 51]=1
        velocities[2, 51]=0
        vehicles[1, 50]=1
        velocities[1, 50]=1
        vehicles[3, 51]=1
        velocities[3, 51]=4
        
    end
end

load_scenario (generic function with 1 method)

In [11]:
for p = 79:83
    if p>80
        tp=p%80
    else
        tp=p
    end
    println(tp)
end

println(ceil(45+1/2)<=47)

79
80
1
2
3
true
